In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('IMDF').getOrCreate()

In [0]:
df = spark.read.csv('dbfs:/FileStore/tables/incident_event_log_reduced.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- number: string (nullable = true)
-- incident_state: string (nullable = true)
-- active: boolean (nullable = true)
-- reassignment_count: integer (nullable = true)
-- reopen_count: integer (nullable = true)
-- sys_mod_count: integer (nullable = true)
-- made_sla: boolean (nullable = true)
-- caller_id: string (nullable = true)
-- opened_by: string (nullable = true)
-- opened_at: string (nullable = true)
-- sys_created_by: string (nullable = true)
-- sys_created_at: string (nullable = true)
-- sys_updated_by: string (nullable = true)
-- sys_updated_at: string (nullable = true)
-- contact_type: string (nullable = true)
-- location: string (nullable = true)
-- category: string (nullable = true)
-- subcategory: string (nullable = true)
-- u_symptom: string (nullable = true)
-- cmdb_ci: string (nullable = true)
-- impact: string (nullable = true)
-- urgency: string (nullable = true)
-- priority: string (nullable = true)
-- assignment_group: string (nullable = true)
-- assigned_to: string (nullable = true)
-- knowledge: boolean (nullable = true)
-- u_priority_confirmation: boolean (nullable = true)
-- notify: string (nullable = true)
-- problem_id: string (nullable = true)
-- rfc: string (nullable = true)
-- vendor: string (nullable = true)
-- caused_by: string (nullable = true)
-- closed_code: string (nullable = true)
-- resolved_by: string (nullable = true)
-- resolved_at: string (nullable = true)
-- closed_at: string (nullable = true)

In [0]:
from pyspark.sql.functions import datediff,date_format,to_date,to_timestamp

In [0]:
df=df.withColumn('resolved_ts',to_timestamp(df.resolved_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('opened_ts',to_timestamp(df.opened_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_created_ts',to_timestamp(df.sys_created_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_updated_ts',to_timestamp(df.sys_updated_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('closed_ts',to_timestamp(df.closed_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('resolved',to_date(df.resolved_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('opened',to_date(df.opened_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_created',to_date(df.sys_created_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('sys_updated',to_date(df.sys_updated_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('closed',to_date(df.closed_at, 'dd/MM/yyyy HH:mm')).\
                withColumn('duration',datediff(to_date(df.resolved_at, 'dd/MM/yyyy HH:mm'),to_date(df.opened_at, 'dd/MM/yyyy HH:mm')))

In [0]:
df_unique_incidents=df.filter("incident_state=='Closed'").sort("sys_mod_count",ascending=False).dropDuplicates(["number"])

#### 1. Top 5 people with most resolved incidents

In [0]:
A1=df_unique_incidents.groupby("resolved_by").count().sort("count",ascending=False)

In [0]:
A1.show(n=5)

+---------------+-----+
 resolved_by|count|
+---------------+-----+
 Resolved by 11| 3071|
 Resolved by 15| 2415|
Resolved by 103| 689|
Resolved by 177| 686|
 Resolved by 32| 597|
+---------------+-----+
only showing top 5 rows

#### 2. Based on least average duration, find the top 5 people with maxmium number of incidents resolved

In [0]:
from pyspark.sql import functions as F

In [0]:
A2= df_unique_incidents.groupby("resolved_by").agg(F.count("duration"),F.mean("duration")).\
    withColumnRenamed("count(duration)","Incidents Resolved").\
    withColumnRenamed("avg(duration)","Average Duration").\
    orderBy(["Average Duration","Incidents Resolved"],ascending=[True,False])

In [0]:
A2.show(n=5)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-699501117654355> in <module> 
 ----> 1 A2 . show ( n = 5 ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 439 """
 440 if isinstance ( truncate , bool ) and truncate : 
 --> 441 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 442 else : 
 443 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o1092.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 140.0 failed 1 times, most recent failure: Lost task 0.0 in stage 140.0 (TID 97, ip-10-172-253-6.us-west-2.compute.internal, executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '29/2/2016 11:29' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.$anonfun$parse$1(TimestampFormatter.scala:86)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:77)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:733)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.util.random.SamplingUtils$.reservoirSampleAndCount(SamplingUtils.scala:41)
	at org.apache.spark.RangePartitioner$.$anonfun$sketch$1(Partitioner.scala:306)
	at org.apache.spark.RangePartitioner$.$anonfun$sketch$1$adapted(Partitioner.scala:304)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsWithIndex$2(RDD.scala:896)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsWithIndex$2$adapted(RDD.scala:896)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:60)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:356)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:320)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:144)
	at org.apache.spark.scheduler.Task.run(Task.scala:117)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun

#### 3. People with maximum number of high impact incidents resolved

In [0]:
A3= df_unique_incidents.select(["resolved_by","impact","duration"]).\
    groupby(["impact","resolved_by"]).count().\
    sort(["impact","count"],ascending=[True,False])

In [0]:
A3.show(n=10)

#### 4a. In each impact levels, find the person with most number of incidents resolved

In [0]:
A4a=df_unique_incidents.select(["resolved_by","impact","duration"]).\
    groupby(["impact","resolved_by"]).count().\
    sort(["impact","count"],ascending=[True,False]).\
    dropDuplicates(["impact"])

In [0]:
A4a.show()

#### 4b. In each urgency levels, find the person with most number of incidents resolved

In [0]:
A4b=df_unique_incidents.select(["resolved_by","urgency","duration"]).\
    groupby(["urgency","resolved_by"]).count().\
    sort(["urgency","count"],ascending=[True,False]).dropDuplicates(["urgency"])

In [0]:
A4b.show()

#### 4c. In each priority levels, find the person with most number of incidents resolved

In [0]:
A4c=df_unique_incidents.select(["resolved_by","priority","duration"]).\
    groupby(["priority","resolved_by"]).count().\
    sort(["priority","count"],ascending=[True,False]).dropDuplicates(["priority"])

In [0]:
A4c.show()

#### 5. Find each contact type as a percentage of total incidents

In [0]:
from pyspark.sql.window import Window

In [0]:
A5= df_unique_incidents.select(["contact_type"]).\
    groupby(["contact_type"]).count().\
    withColumn("percentage",F.round(F.col("count")*100/F.sum("count").over(Window.partitionBy()),2))

In [0]:
A5.show()

+--------------+-----+----------+
 contact_type|count|percentage|
+--------------+-----+----------+
 Phone|24688| 99.08|
 Email| 59| 0.24|
 Self service| 158| 0.63|
 IVR| 9| 0.04|
Direct opening| 4| 0.02|
+--------------+-----+----------+

#### 6. On each priority level, find the percentage of incidents which made SLA and which did not.

In [0]:
A6= df_unique_incidents.select(["priority","made_sla"]).\
    groupby(["priority","made_sla"]).count().\
    withColumnRenamed("count","Population").\
    withColumn("Made SLA %",F.round(F.col("Population")*100/F.sum("Population").over(Window.partitionBy("priority")),2)).\
    sort(["priority","made_sla"],ascending=[True,False])

In [0]:
df_unique_incidents.select(["priority","made_sla"]).groupby(["priority","made_sla"]).count().show()

+------------+--------+-----+
 priority|made_sla|count|
+------------+--------+-----+
 4 - Low| true| 651|
3 - Moderate| false| 8321|
1 - Critical| true| 5|
 4 - Low| false| 123|
1 - Critical| false| 265|
 2 - High| false| 406|
3 - Moderate| true|15145|
 2 - High| true| 2|
+------------+--------+-----+

In [0]:
A6.show()

+------------+--------+----------+----------+
 priority|made_sla|Population|Made SLA %|
+------------+--------+----------+----------+
1 - Critical| true| 5| 1.85|
1 - Critical| false| 265| 98.15|
 2 - High| true| 2| 0.49|
 2 - High| false| 406| 99.51|
3 - Moderate| true| 15145| 64.54|
3 - Moderate| false| 8321| 35.46|
 4 - Low| true| 651| 84.11|
 4 - Low| false| 123| 15.89|
+------------+--------+----------+----------+

#### 7. Top 5 location with the maximum number of incidents reported

In [0]:
A7= df_unique_incidents.groupby(["location"]).agg({"number":"count"}).\
    withColumnRenamed("count(number)","Incidents Reported").sort(["Incidents Reported"],ascending=False)

In [0]:
df_unique_incidents.groupby(["location"]).count().\
withColumnRenamed("count","Incident Reported").sort(["Incident Reported"],ascending=False).show(5)

+------------+-----------------+
 location|Incident Reported|
+------------+-----------------+
Location 204| 5554|
Location 161| 4002|
Location 143| 3276|
Location 108| 2140|
 Location 93| 1934|
+------------+-----------------+
only showing top 5 rows

In [0]:
A7.show(5)

+------------+------------------+
 location|Incidents Reported|
+------------+------------------+
Location 204| 5554|
Location 161| 4002|
Location 143| 3276|
Location 108| 2140|
 Location 93| 1934|
+------------+------------------+
only showing top 5 rows

#### 8. Which category of issues missed meeting the SLA the most?

In [0]:
A8= df_unique_incidents.filter("made_sla==false").groupby(["category"]).\
    agg({"made_sla":"count"}).withColumnRenamed("count(made_sla)","Incidents failed to make SLA").\
    sort(["Incidents failed to make SLA"],ascending=False)

In [0]:
A8.show(5)

+-----------+----------------------------+
 category|Incidents failed to make SLA|
+-----------+----------------------------+
Category 46| 1254|
Category 26| 1017|
Category 53| 1009|
Category 42| 689|
Category 23| 505|
+-----------+----------------------------+
only showing top 5 rows

In [0]:
df_unique_incidents.filter("made_sla==false").groupby(["category"]).count().sort(['count'],ascending=False).show(5)

+-----------+-----+
 category|count|
+-----------+-----+
Category 46| 1254|
Category 26| 1017|
Category 53| 1009|
Category 42| 689|
Category 23| 505|
+-----------+-----+
only showing top 5 rows